In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
sns.set_theme(palette='colorblind')

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# load the bike stations and yelp poi dataframes
bike_stations = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)
# yelp_poi = pd.read_csv('../data/bike_stations_barcelona_poi_yelp.csv', index_col=0)

# Rename bike_stations id column to match yelp_poi so they can be merged
# bike_stations = bike_stations.rename(columns={'id': 'station_id'})

In [ ]:
# Checking the number of rows of each df
print(bike_stations.shape)
# print(yelp_poi.shape)

In [ ]:
# Join both dataframes based on the station_id
full_df = pd.merge(bike_stations, yelp_poi, how='outer', on='station_id')

full_df = full_df.rename(columns={'name_x': 'station_name', 'name_y': 'poi_name'})

full_df.head()

In [ ]:
# Checking if the merge worked correctly
print('Diff of number of bike stations:', bike_stations.shape[0] - len(full_df['station_id'].unique()))

print('Diff of number of POI:', yelp_poi.shape[0] - full_df.shape[0])

In [ ]:
full_df.info()

In [ ]:
# Identify the points of interest that are null
full_df[full_df['poi_id'].isnull()]

In [ ]:
#save data
full_df.to_csv('../data/bike_stations_hamilton_full_poi.csv')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
full_df.head()

In [ ]:
full_df.shape

In [ ]:
full_df.info()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(14, 5))

sns.histplot(ax=axes[0,0], data=full_df, x='empty_slots', kde=True)
sns.histplot(ax=axes[0,1], data=full_df, x='free_bikes', kde=True)
sns.histplot(ax=axes[0,2], data=full_df, x='latitude', kde=True)
sns.histplot(ax=axes[0,3], data=full_df, x='longitude', kde=True)
sns.histplot(ax=axes[1,0], data=full_df, x='review_count', kde=True)
sns.histplot(ax=axes[1,1], data=full_df, x='rating', kde=True)
sns.histplot(ax=axes[1,2], data=full_df, x='price', kde=True)
sns.histplot(ax=axes[1,3], data=full_df, x='distance', kde=True)

fig.tight_layout()

plt.show()

fig.savefig('../images/full_df_hist.svg')

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(12, 8))


sns.boxplot(ax=axes[0, 0], data=full_df, y='empty_slots')
sns.boxplot(ax=axes[0, 1], data=full_df, y='free_bikes')
sns.boxplot(ax=axes[0, 2], data=full_df, y='latitude')
sns.boxplot(ax=axes[0, 3], data=full_df, y='longitude')
sns.boxplot(ax=axes[1, 0], data=full_df, y='review_count')
sns.boxplot(ax=axes[1, 1], data=full_df, y='rating')
sns.boxplot(ax=axes[1, 2], data=full_df, y='price')
sns.boxplot(ax=axes[1, 3], data=full_df, y='distance')


plt.tight_layout()

plt.show()

fig.savefig('../images/full_df_boxplot.svg')

In [ ]:
ax = sns.heatmap(full_df.select_dtypes(exclude=[object]).corr(), vmin=-1, vmax=1, cmap='RdYlGn', annot=True, linewidth=.1, linecolor="black")
ax.set(title='Head Map Full Dataset')

plt.tight_layout()

plt.savefig('../

In [ ]:
sns.pairplot(full_df)

plt.savefig('../images/full_df_pairgrid.svg')
plt.show()

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
connection = create_connection("../data/hamilton_bikes.sqlite")

Look at the data before and after the join to validate your data.

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
bike_stations.info()

In [ ]:
create_bike_stations_table = """
CREATE TABLE IF NOT EXISTS bike_stations_hamilton (
  empty_slots INTEGER,
  free_bikes INTEGER,
  station_id TEXT NOT NULL PRIMARY KEY,
  latitude FLOAT,
  longitude FLOAT,
  name TEXT
);
"""

In [ ]:
execute_query(connection, create_bike_stations_table)

In [ ]:
yelp_poi.info()

In [ ]:
create_yelp_poi_table = """
CREATE TABLE IF NOT EXISTS yelp_poi_hamilton (
  poi_id TEXT,
  name TEXT,
  review_count FLOAT,
  rating FLOAT,
  price TEXT,
  distance FLOAT,
  full_address TEXT,
  station_id TEXT,
  FOREIGN KEY (station_id) REFERENCES bike_stations_hamilton (station_id)
);

In [ ]:
execute_query(connection, create_yelp_poi_table)

In [ ]:
bike_stations.to_sql('bike_stations_hamilton', connection, if_exists='replace')

In [ ]:
yelp_poi.to_sql('yelp_poi_hamilton', connection, if_exists='replace')

In [ ]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
select_stations = "SELECT * from bike_stations_hamilton limit 5"
bike_stations_table = execute_read_query(connection, select_stations)

for station in bike_stations_table:
    print(station)

In [ ]:
select_poi = "SELECT * from yelp_poi_hamilton limit 5"
yelp_poi_table = execute_read_query(connection, select_poi)

for poi in yelp_poi_table:
    print(poi)

In [ ]:
# Checking if the merge worked correctly
print('Diff of number of bike stations:', bike_stations.shape[0] - len(full_df['station_id'].unique()))

print('Diff of number of POI:', yelp_poi.shape[0] - full_df.shape[0])

In [ ]:
full_df.info()